In [1]:
import tensorflow as tf
import keras.preprocessing.image as kpi
import keras.layers as kl
import keras.optimizers as ko
import keras.backend as k
import keras.models as km
import keras.applications as ka

Using TensorFlow backend.


In [2]:
data_dir = 'data'
N_train = 1000
N_val = 400
data_dir_sub = data_dir+'/train_val'

In [3]:
img_width = 150
img_height = 150

In [4]:
#Chargement des données

In [6]:
# Définition du réseau
model = km.Sequential()
#model.add(kl.Dense(512, activation='relu', input_shape=(784,)))
#model.add(kl.Dropout(0.2))
#model.add(kl.Dense(1, activation='softmax'))
model.add(kl.Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3), data_format="channels_last"))
model.add(kl.Activation('relu'))
model.add(kl.MaxPooling2D(pool_size=(2, 2)))

model.add(kl.Conv2D(32, (3, 3)))
model.add(kl.Activation('relu'))
model.add(kl.MaxPooling2D(pool_size=(2, 2)))

#model.add(kl.Flatten())
model.add(kl.Dense(64,name='first', input_shape=(img_width, img_height, 3)))
model.add(kl.Activation('relu'))
model.add(kl.Dropout(0.5))
model.add(kl.Dense(1,name='second'))
model.add(kl.Activation('sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_5 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_6 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
first (Dense)                (None, 36, 36, 64)        2112      
__________

In [7]:
# Compilation
model.compile(loss='binary_crossentropy',
optimizer=ko.RMSprop(),
metrics=['accuracy'])

In [8]:
#Configuration pour le training
train_datagen = kpi.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [9]:
#Configuration pour la validation
valid_datagen = kpi.ImageDataGenerator(rescale=1./255)

In [10]:
#Nous divisons l'ensemble d'entraînement en batchs, 
#chaque epochs passe par tout l'ensemble d'entraînement.
#Chaque itération passe par batch.
batch_size = 20 # 800/20
epochs = 10

In [11]:
#Un générateur qui va lire les images trouvées dans 'datarain_val / train'
train_generator = train_datagen.flow_from_directory(
        data_dir_sub+"/train/",  #le répertoire
        target_size=(img_width, img_height), #toutes les images seront redimensionnées à 150x150
        batch_size=batch_size,
        class_mode='binary',
        classes=['cats','dogs'])

Found 0 images belonging to 2 classes.


In [12]:
#Un générateur similaire, pour les données de validation
validation_generator = valid_datagen.flow_from_directory(
        data_dir_sub+"/validation/",
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')

Found 0 images belonging to 2 classes.


In [ ]:
model.fit_generator(train_generator,
                    steps_per_epoch=N_train// batch_size,
                    epochs=epochs,
                    validation_data=validation_generator,
                    validation_steps=N_val// batch_size)
###################################

Epoch 1/10


In [26]:
model.fit_generator(train_generator,
                    steps_per_epoch=1000/500,
                    epochs=epochs,
                    validation_data=validation_generator,
                    validation_steps=400/20)

Epoch 1/10
2/2 [==============================] - 22s 11s/step - loss: 0.7182 - acc: 0.7250 - val_loss: 0.8718 - val_acc: 0.5675
Epoch 2/10
2/2 [==============================] - 8s 4s/step - loss: 0.9549 - acc: 0.4500 - val_loss: 0.5819 - val_acc: 0.7000
Epoch 3/10
2/2 [==============================] - 7s 4s/step - loss: 0.6045 - acc: 0.7750 - val_loss: 0.5820 - val_acc: 0.7300
Epoch 4/10
2/2 [==============================] - 8s 4s/step - loss: 0.5456 - acc: 0.7250 - val_loss: 0.6000 - val_acc: 0.6575
Epoch 5/10
2/2 [==============================] - 8s 4s/step - loss: 0.6909 - acc: 0.6500 - val_loss: 0.5888 - val_acc: 0.7000
Epoch 6/10
2/2 [==============================] - 8s 4s/step - loss: 0.6080 - acc: 0.6750 - val_loss: 0.5858 - val_acc: 0.7000
Epoch 7/10
2/2 [==============================] - 8s 4s/step - loss: 0.4534 - acc: 0.8750 - val_loss: 0.6011 - val_acc: 0.6900
Epoch 8/10
2/2 [==============================] - 8s 4s/step - loss: 0.6804 - acc: 0.6500 - val_loss: 0.5637 

In [19]:
#fit_generator(self, generator, steps_per_epoch=None,epochs=1, verbose=1, callbacks=None,validation_data=None, validation_steps=None, class_weight=None, max_queue_size=10, workers=1, use_multiprocessing=False, shuffle=True, initial_epoch=0)

model.predict_generator(train_generator, steps=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)

array([[ 0.68677443],
       [ 0.38389698],
       [ 0.96185833],
       ..., 
       [ 0.2896696 ],
       [ 0.21260695],
       [ 0.11691327]], dtype=float32)

In [15]:
model.get_layer(name='first', index=None)

In [16]:
model.get_layer(name='second', index=None)

In [17]:

score_conv_val = model.evaluate_generator(validation_generator, N_val //batch_size)
score_conv_train = model.evaluate_generator(train_generator, N_train// batch_size)

print('Train accuracy:', score_conv_train[1])
print('Test accuracy:', score_conv_val[1])

Train accuracy: 0.712999998331
Test accuracy: 0.642499995232


In [18]:

model_InceptionV3 = ka.InceptionV3(include_top=False, weights='imagenet', input_shape=(150,150,3))
print('Model loaded.')

Model loaded.


In [ ]:
top_model = km.Sequential()
top_model.add(kl.Flatten(input_shape=model_InceptionV3[1:]))
top_model.add(kl.Dense(64, activation='relu'))
top_model.add(kl.Dropout(0.5))
top_model.add(kl.Dense(1, activation='sigmoid'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning

top_model.load_weights(data_dir_sub+'/models_convolutional_network_InceptionV3.h5')